# ETL(EXTRACT, TRANSFORM AND LOAD)- INDIVIDUAL PROJECT - AIRPLANE CRASHES

#### Import Needed Libraries for this project

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import re


#### After fixing dataset structure , next step is to load the dataset

In [2]:
df=pd.read_csv("Dataset_original/AccidentesAviones.csv",index_col=0)



#### Dataset view

In [3]:
df

,fecha,HORA declarada,Ruta,OperadOR,flight_no,route,ac_type,registration,cn_ln,all_aboard,PASAJEROS A BORDO,crew_aboard,cantidad de fallecidos,passenger_fatalities,crew_fatalities,ground,summary
0,"September 17, 1908",1718,"Fort Myer, Virginia",Military - U.S. Army,?,Demonstration,Wright Flyer III,?,1,2,1,1,1,1,0,0,"During a demonstration flight, a U.S. Army fly..."
1,"September 07, 1909",?,"Juvisy-sur-Orge, France",?,?,Air show,Wright Byplane,SC1,?,1,0,1,1,0,0,0,Eugene Lefebvre was the first pilot to ever be...
2,"July 12, 1912",0630,"Atlantic City, New Jersey",Military - U.S. Navy,?,Test flight,Dirigible,?,?,5,0,5,5,0,5,0,First U.S. dirigible Akron exploded just offsh...
3,"August 06, 1913",?,"Victoria, British Columbia, Canada",Private,?,?,Curtiss seaplane,?,?,1,0,1,1,0,1,0,The first fatal airplane accident in Canada oc...
4,"September 09, 1913",1830,Over the North Sea,Military - German Navy,?,?,Zeppelin L-1 (airship),?,?,20,?,?,14,?,?,0,The airship flew into a thunderstorm and encou...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5003,"March 28, 2021",1835,"Near Butte, Alaska",Soloy Helicopters,?,Sightseeing Charter,Eurocopter AS350B3 Ecureuil,N351SH,4598,6,5,1,5,4,1,0,The sightseeing helicopter crashed after missi...
5004,"May 21, 2021",1800,"Near Kaduna, Nigeria",Military - Nigerian Air Force,?,?,Beechcraft B300 King Air 350i,NAF203,FL-891,11,7,4,11,7,4,0,"While on final approach, in poor weather condi..."
5005,"June 10, 2021",0800,"Near Pyin Oo Lwin, Myanmar",Military - Myanmar Air Force,?,Naypyidaw - Anisakan,Beechcraft 1900D,4610,E-325,14,12,2,12,11,1,0,The plane was carrying military personnel and ...
5006,"July 04, 2021",11:30,"Patikul, Sulu, Philippines",Military - Philippine Air Force,?,Cagayan de Oro-Lumbia - Jolo,Lockheed C-130H Hercules,5125,5125,96,88,8,50,?,?,3,"While attempting to land at Jolo Airport, the ..."


#### Litle description of the dataset

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5008 entries, 0 to 5007
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   fecha                   5008 non-null   object
 1   HORA declarada          5008 non-null   object
 2   Ruta                    5008 non-null   object
 3   OperadOR                5008 non-null   object
 4   flight_no               5008 non-null   object
 5   route                   5008 non-null   object
 6   ac_type                 5008 non-null   object
 7   registration            5008 non-null   object
 8   cn_ln                   5008 non-null   object
 9   all_aboard              5008 non-null   object
 10  PASAJEROS A BORDO       5008 non-null   object
 11  crew_aboard             5008 non-null   object
 12  cantidad de fallecidos  5008 non-null   object
 13  passenger_fatalities    5008 non-null   object
 14  crew_fatalities         5008 non-null   object
 15  grou

#### In this dataset null values were filled by "?", so they have to be replaced with NaN values

In [5]:
for i in df.columns:
    df[i].replace("?",np.nan,inplace=True)

#### Going to first colum , they have to be normalized using datetime formatting

In [6]:
for i in range(df.shape[0]):
    df.fecha.iloc[i] = datetime.strptime(df.fecha.iloc[i], "%B %d, %Y")
df['fecha'] = pd.to_datetime(df['fecha']).dt.date


#### Seeing Column: Hora declarada, it shows that it's not cleaned cause it's not only numerical data

In [7]:
df["HORA declarada"][df["HORA declarada"].str.len()>4].value_counts()

10:45      4
08:00      3
17:00      3
23:30      3
10:30      3
          ..
18:30      1
03:41      1
08:18      1
c 10:30    1
0500Z      1
Name: HORA declarada, Length: 172, dtype: int64

#### Clean and normalize the data in this column

In [8]:
df["HORA declarada"]=df["HORA declarada"].astype(str)
for i in range(df.shape[0]):
    df["HORA declarada"].iloc[i] = re.sub(r"[^0-9$]", "", df["HORA declarada"].iloc[i])
df["HORA declarada"]=df["HORA declarada"].replace("",np.nan)
df['HORA declarada'] = pd.to_datetime(df['HORA declarada'], format='%H%M').dt.time


#### Rename columns to get them easier to understand

In [9]:
df.rename(columns={"fecha": "Date", "HORA declarada": "Time", "Ruta": "Location", "OperadOR": "Operator", "route": "Route", "ac_type": "Ac Type", "registration": "Registration", "all_aboard": "All Aboard", "PASAJEROS A BORDO": "Passengers Board", "crew_aboard": "Crew Board", "cantidad de fallecidos": "All Fatalities", "passenger_fatalities": "Passenger Fatalities","crew_fatalities":"Crew fatalities","ground":"Ground","summary":"Summary"},inplace=True)

#### Normalization Location to Country 

In [18]:
df["Country"]=df.Location
for i in range(df.shape[0]):
    if isinstance(df.Location.iloc[i],str):
        df["Country"].iloc[i]=df.Location.iloc[i].split(", ")[-1]
df.Country=df.Country.str.strip()

#### Normalization USA States to Country

In [19]:
estados = ["Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado",
           "Connecticut", "Delaware", "Florida", "Georgia", "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa",
           "Kansas", "Kentucky", "Louisiana", "Maine", "Maryland", "Massachusetts", "Michigan",
           "Minnesota", "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", "New Hampshire", "New Jersey", "New Mexico", "New York", "North Carolina", "North Dakota", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington", "West Virginia", "Wisconsin", "Wyoming"]
for i in range(df.shape[0]):
    if isinstance(df.Country.iloc[i],str):
        if df.Country.iloc[i] in estados:
            df["Country"].iloc[i]="United States of America"

#### View Null Values in People

In [24]:
df[df["All Aboard"].isna()]

,Date,Time,Location,Operator,flight_no,Route,Ac Type,Registration,cn_ln,All Aboard,Passengers Board,Crew Board,All Fatalities,Passenger Fatalities,Crew fatalities,Ground,Summary,Country
26,1919-07-21,16:55:00,"Chicago, Illinois",Wingfoot Air Express Goodyear Tire,NaN,Shuttle,FD Type Dirigible,NaN,NaN,NaN,NaN,NaN,3,2,1,10,"The dirigible, cruising at 1,200 ft. caught fi...",United States of America
103,1925-09-07,NaT,"Toul, France",CIDNA,NaN,NaN,Spad 33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,France
347,1934-08-10,NaT,"Ningbo, China",China National Aviation Corporation,NaN,NaN,Sikorsky S-38B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,China
384,1935-08-13,NaT,"Hangow, China",China National Aviation Corporation,NaN,NaN,Sikorsky S-38B,NV40V,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Destoryed in a storm.,China
451,1936-12-26,NaT,"Nanking, China",China National Aviation Corporation,NaN,NaN,Douglas DC-2,NC14269,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,China
652,1942-10-01,NaT,"Kunming, China",China National Aviation Corporation,NaN,NaN,Douglas C-47,69,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Crashed while attempting to land after losing ...,China
754,1944-11-09,NaT,"Seljord, Norway",Military - U.S. Army Air Corps,NaN,NaN,NaN,42-52196,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Norway
854,1946-03-18,NaT,"Between Chungking and Shanghai, China",China National Aviation Corporation,NaN,Chunking - Shanghai,NaN,139,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Disappeared while en route. Plane never located.,China
1231,1951-01-20,NaT,"Huilyo, Peru",Military - TAM,NaN,NaN,Douglas DC-3,NaN,NaN,NaN,NaN,NaN,16,NaN,NaN,NaN,NaN,Peru
1362,1952-12-22,NaT,"Damascus, Syria",Syrian Air,NaN,NaN,Douglas DC-3,NaN,NaN,NaN,NaN,NaN,9,NaN,NaN,NaN,NaN,Syria


#### Final dataset view after ETL analysis

In [22]:
df

,Date,Time,Location,Operator,flight_no,Route,Ac Type,Registration,cn_ln,All Aboard,Passengers Board,Crew Board,All Fatalities,Passenger Fatalities,Crew fatalities,Ground,Summary,Country
0,1908-09-17,17:18:00,"Fort Myer, Virginia",Military - U.S. Army,NaN,Demonstration,Wright Flyer III,NaN,1,2,1,1,1,1,0,0,"During a demonstration flight, a U.S. Army fly...",United States of America
1,1909-09-07,NaT,"Juvisy-sur-Orge, France",NaN,NaN,Air show,Wright Byplane,SC1,NaN,1,0,1,1,0,0,0,Eugene Lefebvre was the first pilot to ever be...,France
2,1912-07-12,06:30:00,"Atlantic City, New Jersey",Military - U.S. Navy,NaN,Test flight,Dirigible,NaN,NaN,5,0,5,5,0,5,0,First U.S. dirigible Akron exploded just offsh...,United States of America
3,1913-08-06,NaT,"Victoria, British Columbia, Canada",Private,NaN,NaN,Curtiss seaplane,NaN,NaN,1,0,1,1,0,1,0,The first fatal airplane accident in Canada oc...,Canada
4,1913-09-09,18:30:00,Over the North Sea,Military - German Navy,NaN,NaN,Zeppelin L-1 (airship),NaN,NaN,20,NaN,NaN,14,NaN,NaN,0,The airship flew into a thunderstorm and encou...,Over the North Sea
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5003,2021-03-28,18:35:00,"Near Butte, Alaska",Soloy Helicopters,NaN,Sightseeing Charter,Eurocopter AS350B3 Ecureuil,N351SH,4598,6,5,1,5,4,1,0,The sightseeing helicopter crashed after missi...,United States of America
5004,2021-05-21,18:00:00,"Near Kaduna, Nigeria",Military - Nigerian Air Force,NaN,NaN,Beechcraft B300 King Air 350i,NAF203,FL-891,11,7,4,11,7,4,0,"While on final approach, in poor weather condi...",Nigeria
5005,2021-06-10,08:00:00,"Near Pyin Oo Lwin, Myanmar",Military - Myanmar Air Force,NaN,Naypyidaw - Anisakan,Beechcraft 1900D,4610,E-325,14,12,2,12,11,1,0,The plane was carrying military personnel and ...,Myanmar
5006,2021-07-04,11:30:00,"Patikul, Sulu, Philippines",Military - Philippine Air Force,NaN,Cagayan de Oro-Lumbia - Jolo,Lockheed C-130H Hercules,5125,5125,96,88,8,50,NaN,NaN,3,"While attempting to land at Jolo Airport, the ...",Philippines


#### Export dataset after ETL analysis

In [21]:
df.to_csv("Dataset_final/AccidentesAvionesETL.csv")